# Finetune a YOLO model using a dataset from OpenStreetMap data and Mapbox tiles

## Dependencies

In [ ]:
%pip install --quiet ultralytics

## Download dataset from HuggingFace Hub Datasets

You need to set the `HF_TOKEN` Colab secret:

- Create an account: https://huggingface.co/join
- Follow this guide about [`User Access Tokens`](https://huggingface.co/docs/hub/security-tokens)

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
REPO = "mozilla-ai/osm-swimming-pools"

In [ ]:
hf_hub_download(REPO, filename="train.zip", repo_type="dataset", local_dir="datasets")

In [ ]:
hf_hub_download(REPO, filename="val.zip", repo_type="dataset", local_dir="datasets")

In [ ]:
hf_hub_download(
    REPO, filename="yolo_dataset.yaml", repo_type="dataset", local_dir="datasets"
)

In [ ]:
!unzip -q datasets/train.zip

In [ ]:
!unzip -q datasets/val.zip

# Finetune model

In [ ]:
from ultralytics import YOLO

Check the [available models](https://docs.ultralytics.com/tasks/detect/#models).

In [ ]:
MODEL = "yolo11m.pt"

In [ ]:
yolo = YOLO(MODEL)

In [ ]:
yolo.train(
    data="datasets/yolo_dataset.yaml",
    patience=3,
    imgsz=512,
    scale=0.0,
    flipud=0.5,
    translate=0.0,
)

# Check results

In [ ]:
from PIL import Image

In [ ]:
Image.open("/content/runs/detect/train/val_batch0_labels.jpg")

In [ ]:
Image.open("/content/runs/detect/train/val_batch0_pred.jpg")

# Upload model

In [ ]:
USER = "daavoo"
REPO = "yolo-osm-pool-detector"

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

In [ ]:
api = HfApi()

In [ ]:
api.create_repo(f"{USER}/{REPO}", token=userdata.get("HF_TOKEN"), repo_type="model")

In [ ]:
api.upload_file(
    token=userdata.get("HF_TOKEN"),
    path_or_fileobj="/content/runs/detect/train/weights/best.pt",
    path_in_repo="model.pt",
    repo_id=f"{USER}/{REPO}",
    repo_type="model",
)